In [ ]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras

import datetime
import keras
from keras.layers import Conv2D, MaxPooling2D, Input, Concatenate, UpSampling2D, BatchNormalization
from keras.models import Model

## Convolutional Blocks

In [ ]:
# encoded layers ---> conv
# maxpooled layers ---> pool
def encoder(layer, filters, kernel_size = (3,3), padding = "same", strides = 1):
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(layer)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.2)(conv)
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(conv)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.2)(conv)    
    
    pool = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')(conv)
    
    return conv, pool



def bottleneck(layer, filters, kernel_size = (3,3), padding = "same", strides = 1):
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(layer)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.2)(conv)
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(conv)
    conv = BatchNormalization()(conv)
    conv = Dropout(0.2)(conv)
    
    return conv



# decoded layers ---> conv
def decoder(concatenated_layer, encoded_layer, filters, kernel_size = (3,3), padding = "same", strides = 1):
    
    up = UpSampling2D((2,2))(concatenated_layer)
    concat = Concatenate()([up,encoded_layer])
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(concat)
    conv = BatchNormalization()(conv)
    
    conv = Conv2D(filters, kernel_size, padding = padding, strides = strides, activation = "relu")(conv)
    conv = BatchNormalization()(conv)
    
    return conv


## U-Net Model

In [ ]:
def Unet():
    
    filter_size = 16
    depth = 4
    
    inputs = Input(shape = (image_size, image_size, 3))
    
    p0 = inputs
    layers = []
    temp = p0
    
    for i in range(depth):
        conv, pooled_filter = encoder(temp, filter_size)
        layers.append(conv)
        
        temp = pooled_filter
        filter_size *= 2
    
    bn = bottleneck(temp, filter_size)
    temp = bn
    
    for layer in layers[::-1]:
        up = decoder(temp, layer, filter_size/2)
        temp = up
        
    outputs = Conv2D(1, (1,1), padding = "same", activation = "sigmoid")(temp)
    del temp
    
    model = Model(inputs, outputs)
    
    return model

In [ ]:
model = Unet()

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.metrics.BinaryCrossentropy()
metrics = tf.metrics.Accuracy()

model.compile(optimizer = optimizer, loss = loss, metrics = metrics)